# 3. Modeling avec GridSearchCV

Ce notebook entraîne plusieurs modèles de classification, optimise leurs hyperparamètres avec GridSearchCV, compare leurs performances et sélectionne automatiquement le meilleur modèle

## Import des bibliothèques

In [ ]:
import pandas as pd
import numpy as np
import pickle
import time
from datetime import datetime

# Modèles de classification
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

# GridSearchCV pour optimisation
from sklearn.model_selection import GridSearchCV

# Métriques
from sklearn.metrics import (
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score,
    classification_report,
    confusion_matrix,
    make_scorer
)

import warnings
warnings.filterwarnings('ignore')

print("Bibliothèques importées avec succès")

Bibliothèques importées avec succès


## Chargement des features

In [ ]:
# Charger les ensembles de features
with open('pkl/feature_sets.pkl', 'rb') as f:
    feature_sets = pickle.load(f)

# Extraire les features et labels
X_train_tfidf = feature_sets['X_train_tfidf']
X_test_tfidf = feature_sets['X_test_tfidf']
X_train_count = feature_sets['X_train_count']
X_test_count = feature_sets['X_test_count']
X_train_svd = feature_sets['X_train_svd']
X_test_svd = feature_sets['X_test_svd']
X_train_combined = feature_sets['X_train_combined']
X_test_combined = feature_sets['X_test_combined']

y_train = feature_sets['y_train']
y_test = feature_sets['y_test']
label_encoder = feature_sets['label_encoder']

print("Features chargées avec succès")
print(f"Nombre de classes: {len(label_encoder.classes_)}")

Features chargées avec succès
Nombre de classes: 119


## Définition des modèles (sans Gradient Boosting)

In [5]:
# Définir les modèles de base (sans optimisation)
base_models = {
    'Logistic_Regression': LogisticRegression(
        max_iter=1000, 
        random_state=42,
        n_jobs=-1,
        solver='saga'
    ),
    'Multinomial_NB': MultinomialNB(),
    'Linear_SVC': LinearSVC(
        max_iter=1000, 
        random_state=42
    ),
    'Random_Forest': RandomForestClassifier(
        n_estimators=100,
        max_depth=20,
        random_state=42,
        n_jobs=-1
    ),
    'KNN': KNeighborsClassifier(
        n_neighbors=5,
        n_jobs=-1
    ),
    'Decision_Tree': DecisionTreeClassifier(
        max_depth=20,
        random_state=42
    )
}

print(f"Nombre de modèles définis: {len(base_models)}")
print(f"Modèles: {list(base_models.keys())}")

Nombre de modèles définis: 6
Modèles: ['Logistic_Regression', 'Multinomial_NB', 'Linear_SVC', 'Random_Forest', 'KNN', 'Decision_Tree']


## Définition des grilles d'hyperparamètres pour GridSearchCV

In [6]:
# Grilles d'hyperparamètres pour chaque modèle
param_grids = {
    'Logistic_Regression': {
        'C': [0.1, 1, 10, 100],
        'solver': ['saga', 'liblinear'],
        'max_iter': [1000, 2000]
    },
    'Multinomial_NB': {
        'alpha': [0.01, 0.1, 0.5, 1.0, 2.0]
    },
    'Linear_SVC': {
        'C': [0.1, 1, 10, 100],
        'max_iter': [1000, 2000]
    },
    'Random_Forest': {
        'n_estimators': [50, 100, 200],
        'max_depth': [10, 20, 30],
        'min_samples_split': [2, 5]
    },
    'KNN': {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance']
    },
    'Decision_Tree': {
        'max_depth': [10, 20, 30, None],
        'min_samples_split': [2, 5, 10]
    }
}

print("Grilles d'hyperparamètres définies:")
for model_name, params in param_grids.items():
    n_combinations = np.prod([len(v) for v in params.values()])
    print(f"  {model_name}: {n_combinations} combinaisons")

Grilles d'hyperparamètres définies:
  Logistic_Regression: 16 combinaisons
  Multinomial_NB: 5 combinaisons
  Linear_SVC: 8 combinaisons
  Random_Forest: 18 combinaisons
  KNN: 8 combinaisons
  Decision_Tree: 12 combinaisons


In [7]:
# Définir les configurations de features
feature_configurations = {
    'TF-IDF': (X_train_tfidf, X_test_tfidf),
    'Count': (X_train_count, X_test_count),
    'SVD': (X_train_svd, X_test_svd),
    'Combined': (X_train_combined, X_test_combined)
}

# Modèles à ignorer pour Combined (nécessitent normalisation)
skip_combined = ['Logistic_Regression', 'Linear_SVC', 'KNN']

print(f"\nNombre de configurations de features: {len(feature_configurations)}")
print(f"Configurations: {list(feature_configurations.keys())}")
print(f"\nModèles ignorés pour 'Combined': {skip_combined}")


Nombre de configurations de features: 4
Configurations: ['TF-IDF', 'Count', 'SVD', 'Combined']

Modèles ignorés pour 'Combined': ['Logistic_Regression', 'Linear_SVC', 'KNN']


## Fonction d'évaluation avec GridSearchCV

In [8]:
def evaluate_model_with_gridsearch(model, param_grid, X_train, X_test, y_train, y_test, model_name):
    """
    Entraîne et évalue un modèle avec GridSearchCV
    Retourne les métriques, le meilleur modèle et les meilleurs paramètres
    """
    print(f"    🔍 GridSearchCV en cours...")
    
    # Définir le scorer
    scorer = make_scorer(f1_score, average='weighted', zero_division=0)
    
    # GridSearchCV
    start_time = time.time()
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=3,  # 3-fold cross-validation
        scoring=scorer,
        n_jobs=-1,
        verbose=0
    )
    
    grid_search.fit(X_train, y_train)
    training_time = time.time() - start_time
    
    # Meilleur modèle
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_
    best_cv_score = grid_search.best_score_
    
    # Prédictions sur le test
    start_time = time.time()
    y_pred = best_model.predict(X_test)
    prediction_time = time.time() - start_time
    
    # Calcul des métriques
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision_weighted': precision_score(y_test, y_pred, average='weighted', zero_division=0),
        'recall_weighted': recall_score(y_test, y_pred, average='weighted', zero_division=0),
        'f1_weighted': f1_score(y_test, y_pred, average='weighted', zero_division=0),
        'precision_macro': precision_score(y_test, y_pred, average='macro', zero_division=0),
        'recall_macro': recall_score(y_test, y_pred, average='macro', zero_division=0),
        'f1_macro': f1_score(y_test, y_pred, average='macro', zero_division=0),
        'training_time': training_time,
        'prediction_time': prediction_time,
        'best_cv_score': best_cv_score
    }
    
    return metrics, best_model, y_pred, best_params

print("Fonction d'évaluation avec GridSearchCV définie")

Fonction d'évaluation avec GridSearchCV définie


## Entraînement et évaluation de tous les modèles avec GridSearchCV

In [9]:
# Dictionnaire pour stocker tous les résultats
all_results = []
trained_models = {}
best_params_dict = {}

print("="*80)
print("DÉBUT DE L'ENTRAÎNEMENT DES MODÈLES AVEC GRIDSEARCHCV")
print("="*80)
print("\n⚠️  NOTE: GridSearchCV prend plus de temps mais optimise les hyperparamètres")
print("    Chaque modèle teste plusieurs combinaisons de paramètres avec CV 3-fold\n")

total_combinations = sum(
    1 for feature_name in feature_configurations 
    for model_name in base_models 
    if not (feature_name == 'Combined' and model_name in skip_combined)
)
current_combination = 0

# Boucle sur toutes les combinaisons modèle-features
for feature_name, (X_train_feat, X_test_feat) in feature_configurations.items():
    print(f"\n{'='*80}")
    print(f"CONFIGURATION DE FEATURES: {feature_name}")
    print(f"{'='*80}")
    
    for model_name, model in base_models.items():
        
        # Ignorer Combined pour certains modèles
        if feature_name == 'Combined' and model_name in skip_combined:
            print(f"\n⏭️  [{model_name}] Ignoré pour {feature_name} (nécessite normalisation)")
            continue
        
        # Ignorer SVD pour Multinomial_NB (valeurs négatives)
        if feature_name == 'SVD' and model_name == 'Multinomial_NB':
            print(f"\n⏭️  [{model_name}] Ignoré pour {feature_name} (valeurs négatives)")
            continue
        
        current_combination += 1
        print(f"\n[{current_combination}/{total_combinations}] Entraînement: {model_name} avec {feature_name}")
        
        try:
            # Évaluer le modèle avec GridSearchCV
            metrics, trained_model, y_pred, best_params = evaluate_model_with_gridsearch(
                model, param_grids[model_name], X_train_feat, X_test_feat, y_train, y_test, model_name
            )
            
            # Stocker les résultats
            result = {
                'model_name': model_name,
                'feature_config': feature_name,
                'combination': f"{model_name}_{feature_name}",
                **metrics
            }
            all_results.append(result)
            
            # Stocker le modèle entraîné
            model_key = f"{model_name}_{feature_name}"
            trained_models[model_key] = {
                'model': trained_model,
                'metrics': metrics,
                'predictions': y_pred
            }
            
            # Stocker les meilleurs paramètres
            best_params_dict[model_key] = best_params
            
            # Afficher les résultats
            print(f"    ✓ Meilleurs paramètres: {best_params}")
            print(f"    ✓ CV Score: {metrics['best_cv_score']:.4f}")
            print(f"    ✓ Test Accuracy: {metrics['accuracy']:.4f}")
            print(f"    ✓ Test F1-Score (weighted): {metrics['f1_weighted']:.4f}")
            print(f"    ✓ Test F1-Score (macro): {metrics['f1_macro']:.4f}")
            print(f"    ✓ Temps d'entraînement: {metrics['training_time']:.2f}s")
            
        except Exception as e:
            print(f"    ❌ ERREUR: {str(e)}")
            continue

print(f"\n{'='*80}")
print("ENTRAÎNEMENT TERMINÉ")
print(f"{'='*80}")

DÉBUT DE L'ENTRAÎNEMENT DES MODÈLES AVEC GRIDSEARCHCV

⚠️  NOTE: GridSearchCV prend plus de temps mais optimise les hyperparamètres
    Chaque modèle teste plusieurs combinaisons de paramètres avec CV 3-fold


CONFIGURATION DE FEATURES: TF-IDF

[1/21] Entraînement: Logistic_Regression avec TF-IDF
    🔍 GridSearchCV en cours...
    ✓ Meilleurs paramètres: {'C': 100, 'max_iter': 1000, 'solver': 'saga'}
    ✓ CV Score: 0.7185
    ✓ Test Accuracy: 0.7337
    ✓ Test F1-Score (weighted): 0.7329
    ✓ Test F1-Score (macro): 0.7243
    ✓ Temps d'entraînement: 133.68s

[2/21] Entraînement: Multinomial_NB avec TF-IDF
    🔍 GridSearchCV en cours...
    ✓ Meilleurs paramètres: {'alpha': 0.1}
    ✓ CV Score: 0.6176
    ✓ Test Accuracy: 0.6463
    ✓ Test F1-Score (weighted): 0.6363
    ✓ Test F1-Score (macro): 0.6436
    ✓ Temps d'entraînement: 0.68s

[3/21] Entraînement: Linear_SVC avec TF-IDF
    🔍 GridSearchCV en cours...
    ✓ Meilleurs paramètres: {'C': 10, 'max_iter': 1000}
    ✓ CV Score: 0.7

## Comparaison des performances

In [10]:
# Créer un DataFrame avec tous les résultats
results_df = pd.DataFrame(all_results)

# Trier par F1-score weighted (métrique principale)
results_df = results_df.sort_values('f1_weighted', ascending=False)

print("\n" + "="*80)
print("COMPARAISON DES PERFORMANCES (Top 10)")
print("="*80)
print(results_df[[
    'combination', 'accuracy', 'f1_weighted', 'f1_macro', 'best_cv_score'
]].head(10).to_string(index=False))


COMPARAISON DES PERFORMANCES (Top 10)
               combination  accuracy  f1_weighted  f1_macro  best_cv_score
    Random_Forest_Combined  0.766260     0.755572  0.734941       0.708508
       Random_Forest_Count  0.762195     0.751029  0.752502       0.709097
 Logistic_Regression_Count  0.741870     0.740510  0.729349       0.727755
      Random_Forest_TF-IDF  0.752033     0.739744  0.717458       0.697167
   Logistic_Regression_SVD  0.733740     0.733250  0.733212       0.708731
Logistic_Regression_TF-IDF  0.733740     0.732886  0.724259       0.718489
          Linear_SVC_Count  0.725610     0.722106  0.699014       0.728166
            Linear_SVC_SVD  0.727642     0.720357  0.713030       0.699651
         Linear_SVC_TF-IDF  0.721545     0.716254  0.709509       0.726087
         Random_Forest_SVD  0.686992     0.686258  0.704231       0.654616


## Sélection automatique du meilleur modèle

In [12]:
# Sélectionner le meilleur modèle basé sur le F1-score weighted
best_result = results_df.iloc[0]
best_model_key = best_result['combination']
best_model_info = trained_models[best_model_key]
best_params = best_params_dict[best_model_key]

print("="*80)
print("MEILLEUR MODÈLE SÉLECTIONNÉ AUTOMATIQUEMENT (OPTIMISÉ AVEC GRIDSEARCHCV)")
print("="*80)
print(f"\nCombination: {best_result['combination']}")
print(f"Modèle: {best_result['model_name']}")
print(f"Features: {best_result['feature_config']}")
print(f"\nMEILLEURS HYPERPARAMÈTRES:")
for param, value in best_params.items():
    print(f"  {param}: {value}")
print(f"\nMÉTRIQUES DE PERFORMANCE:")
print(f"  CV Score (3-fold): {best_result['best_cv_score']:.4f}")
print(f"  Test Accuracy: {best_result['accuracy']:.4f}")
print(f"  Test Precision (weighted): {best_result['precision_weighted']:.4f}")
print(f"  Test Recall (weighted): {best_result['recall_weighted']:.4f}")
print(f"  Test F1-Score (weighted): {best_result['f1_weighted']:.4f}")
print(f"  Test F1-Score (macro): {best_result['f1_macro']:.4f}")
print(f"\nTEMPS:")
print(f"  Entraînement (avec GridSearch): {best_result['training_time']:.2f}s")
print(f"  Prédiction: {best_result['prediction_time']:.4f}s")

MEILLEUR MODÈLE SÉLECTIONNÉ AUTOMATIQUEMENT (OPTIMISÉ AVEC GRIDSEARCHCV)

Combination: Random_Forest_Combined
Modèle: Random_Forest
Features: Combined

MEILLEURS HYPERPARAMÈTRES:
  max_depth: 30
  min_samples_split: 2
  n_estimators: 200

MÉTRIQUES DE PERFORMANCE:
  CV Score (3-fold): 0.7085
  Test Accuracy: 0.7663
  Test Precision (weighted): 0.7876
  Test Recall (weighted): 0.7663
  Test F1-Score (weighted): 0.7556
  Test F1-Score (macro): 0.7349

TEMPS:
  Entraînement (avec GridSearch): 38.15s
  Prédiction: 0.4139s


## Rapport détaillé pour le meilleur modèle

In [ ]:
# Générer le rapport de classification pour le meilleur modèle
best_predictions = best_model_info['predictions']

print("\n" + "="*80)
print("RAPPORT DE CLASSIFICATION DÉTAILLÉ (Meilleur Modèle Optimisé)")
print("="*80)

# Ne garder que les classes réellement présentes dans y_test ou dans les prédictions
present_labels = np.unique(np.concatenate([y_test.values, best_predictions]))
target_names = label_encoder.inverse_transform(present_labels)

print(classification_report(
    y_test,
    best_predictions,
    labels=present_labels,
    target_names=target_names,
    zero_division=0
))


RAPPORT DE CLASSIFICATION DÉTAILLÉ (Meilleur Modèle Optimisé)
                                              precision    recall  f1-score   support

                               AI Researcher       0.80      0.80      0.80         5
                        Academic Coordinator       1.00      1.00      1.00         3
                           Account Executive       1.00      1.00      1.00         2
                             Account Manager       1.00      1.00      1.00         2
                                  Accountant       0.46      0.75      0.57         8
                    Administrative Assistant       1.00      1.00      1.00         3
                                Art Director       0.00      0.00      0.00         3
                                    Attorney       1.00      1.00      1.00         4
                                  BI Analyst       0.83      1.00      0.91         5
                           Backend Developer       0.33      0.25      0.29 

## Tableau des meilleurs hyperparamètres par modèle

In [15]:
print("\n" + "="*80)
print("MEILLEURS HYPERPARAMÈTRES TROUVÉS PAR GRIDSEARCHCV")
print("="*80)

for model_key, params in best_params_dict.items():
    model_result = results_df[results_df['combination'] == model_key].iloc[0]
    print(f"\n{model_key}:")
    print(f"  F1-Score: {model_result['f1_weighted']:.4f}")
    print(f"  Paramètres:")
    for param, value in params.items():
        print(f"    - {param}: {value}")


MEILLEURS HYPERPARAMÈTRES TROUVÉS PAR GRIDSEARCHCV

Logistic_Regression_TF-IDF:
  F1-Score: 0.7329
  Paramètres:
    - C: 100
    - max_iter: 1000
    - solver: saga

Multinomial_NB_TF-IDF:
  F1-Score: 0.6363
  Paramètres:
    - alpha: 0.1

Linear_SVC_TF-IDF:
  F1-Score: 0.7163
  Paramètres:
    - C: 10
    - max_iter: 1000

Random_Forest_TF-IDF:
  F1-Score: 0.7397
  Paramètres:
    - max_depth: 30
    - min_samples_split: 2
    - n_estimators: 200

KNN_TF-IDF:
  F1-Score: 0.6238
  Paramètres:
    - n_neighbors: 9
    - weights: distance

Decision_Tree_TF-IDF:
  F1-Score: 0.6820
  Paramètres:
    - max_depth: None
    - min_samples_split: 5

Logistic_Regression_Count:
  F1-Score: 0.7405
  Paramètres:
    - C: 10
    - max_iter: 1000
    - solver: liblinear

Multinomial_NB_Count:
  F1-Score: 0.6433
  Paramètres:
    - alpha: 1.0

Linear_SVC_Count:
  F1-Score: 0.7221
  Paramètres:
    - C: 0.1
    - max_iter: 1000

Random_Forest_Count:
  F1-Score: 0.7510
  Paramètres:
    - max_depth: 3

## Analyse par type de modèle

In [16]:
# Performance moyenne par modèle
print("\n" + "="*80)
print("PERFORMANCE MOYENNE PAR TYPE DE MODÈLE (APRÈS OPTIMISATION)")
print("="*80)

model_performance = results_df.groupby('model_name')[[
    'accuracy', 'f1_weighted', 'f1_macro'
]].agg(['mean', 'max']).round(4)

print(model_performance)


PERFORMANCE MOYENNE PAR TYPE DE MODÈLE (APRÈS OPTIMISATION)
                    accuracy         f1_weighted         f1_macro        
                        mean     max        mean     max     mean     max
model_name                                                               
Decision_Tree         0.6245  0.6850      0.6235  0.6822   0.6272  0.6932
KNN                   0.6057  0.6382      0.5977  0.6316   0.6139  0.6373
Linear_SVC            0.7249  0.7276      0.7196  0.7221   0.7072  0.7130
Logistic_Regression   0.7364  0.7419      0.7355  0.7405   0.7289  0.7332
Multinomial_NB        0.6477  0.6585      0.6375  0.6433   0.6446  0.6606
Random_Forest         0.7419  0.7663      0.7332  0.7556   0.7273  0.7525


In [17]:
# Performance moyenne par configuration de features
print("\n" + "="*80)
print("PERFORMANCE MOYENNE PAR CONFIGURATION DE FEATURES")
print("="*80)

feature_performance = results_df.groupby('feature_config')[[
    'accuracy', 'f1_weighted', 'f1_macro'
]].agg(['mean', 'max']).round(4)

print(feature_performance)


PERFORMANCE MOYENNE PAR CONFIGURATION DE FEATURES
               accuracy         f1_weighted         f1_macro        
                   mean     max        mean     max     mean     max
feature_config                                                      
Combined         0.6965  0.7663      0.6902  0.7556   0.6920  0.7349
Count            0.6850  0.7622      0.6778  0.7510   0.6734  0.7525
SVD              0.6467  0.7337      0.6459  0.7333   0.6519  0.7332
TF-IDF           0.6961  0.7520      0.6885  0.7397   0.6876  0.7243


## Comparaison: Amélioration apportée par GridSearchCV

In [18]:
print("\n" + "="*80)
print("AMÉLIORATION CV vs TEST")
print("="*80)

# Comparer CV score et test score
comparison_df = results_df[['combination', 'best_cv_score', 'f1_weighted']].copy()
comparison_df['difference'] = comparison_df['f1_weighted'] - comparison_df['best_cv_score']
comparison_df = comparison_df.sort_values('f1_weighted', ascending=False)

print("\nTop 10 modèles - Comparaison CV vs Test:")
print(comparison_df.head(10).to_string(index=False))

print("\n📊 Statistiques:")
print(f"  Différence moyenne (Test - CV): {comparison_df['difference'].mean():.4f}")
print(f"  Différence médiane: {comparison_df['difference'].median():.4f}")
print(f"  Écart-type: {comparison_df['difference'].std():.4f}")


AMÉLIORATION CV vs TEST

Top 10 modèles - Comparaison CV vs Test:
               combination  best_cv_score  f1_weighted  difference
    Random_Forest_Combined       0.708508     0.755572    0.047064
       Random_Forest_Count       0.709097     0.751029    0.041931
 Logistic_Regression_Count       0.727755     0.740510    0.012755
      Random_Forest_TF-IDF       0.697167     0.739744    0.042577
   Logistic_Regression_SVD       0.708731     0.733250    0.024519
Logistic_Regression_TF-IDF       0.718489     0.732886    0.014397
          Linear_SVC_Count       0.728166     0.722106   -0.006060
            Linear_SVC_SVD       0.699651     0.720357    0.020706
         Linear_SVC_TF-IDF       0.726087     0.716254   -0.009833
         Random_Forest_SVD       0.654616     0.686258    0.031642

📊 Statistiques:
  Différence moyenne (Test - CV): 0.0292
  Différence médiane: 0.0254
  Écart-type: 0.0208


## Sauvegarde des résultats et modèles

In [ ]:
# Sauvegarder tous les résultats
modeling_results = {
    'all_results': results_df,
    'trained_models': trained_models,
    'best_params': best_params_dict,
    'best_model_key': best_model_key,
    'best_model': best_model_info['model'],
    'best_metrics': best_model_info['metrics'],
    'best_predictions': best_predictions,
    'best_hyperparams': best_params,
    'label_encoder': label_encoder,
    'y_test': y_test,
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'gridsearch_used': True
}

with open('pkl/modeling_results_gridsearch.pkl', 'wb') as f:
    pickle.dump(modeling_results, f)

print("Résultats du modeling avec GridSearch sauvegardés: pkl/modeling_results_gridsearch.pkl")

Résultats du modeling avec GridSearch sauvegardés: modeling_results_gridsearch.pkl


## Résumé du Modeling avec GridSearchCV

In [ ]:
print("\n" + "="*80)
print("RÉSUMÉ DU MODELING AVEC GRIDSEARCHCV")
print("="*80)

print(f"\n1. MODÈLES ENTRAÎNÉS:")
print(f"   - Nombre de types de modèles: {len(base_models)}")
print(f"   - Nombre de configurations de features: {len(feature_configurations)}")
print(f"   - Total de combinaisons testées: {len(results_df)}")
print(f"   - Modèles ignorés pour 'Combined': {len(skip_combined)}")

print(f"\n2. OPTIMISATION GRIDSEARCHCV:")
print(f"   - Cross-validation: 3-fold")
print(f"   - Métrique d'optimisation: F1-Score weighted")
print(f"   - Hyperparamètres optimisés pour chaque modèle: Oui")

print(f"\n3. MEILLEUR MODÈLE:")
print(f"   - Nom: {best_result['model_name']}")
print(f"   - Features: {best_result['feature_config']}")
print(f"   - CV F1-Score: {best_result['best_cv_score']:.4f}")
print(f"   - Test F1-Score (weighted): {best_result['f1_weighted']:.4f}")
print(f"   - Test Accuracy: {best_result['accuracy']:.4f}")

print(f"\n4. TOP 3 MODÈLES:")
for idx, row in results_df.head(3).iterrows():
    print(f"   {idx + 1}. {row['combination']} - F1: {row['f1_weighted']:.4f}")

print(f"\n5. FICHIERS SAUVEGARDÉS:")
print(f"   - pkl/modeling_results_gridsearch.pkl (tous les modèles et résultats)")

print(f"\n6. AMÉLIORATION vs MODÈLES DE BASE:")
print(f"   - GridSearchCV a optimisé les hyperparamètres")
print(f"   - Cross-validation assure la généralisation")
print(f"   - Performance validée sur ensemble de test")

print("\n" + "="*80)
print("MODELING AVEC GRIDSEARCHCV TERMINÉ AVEC SUCCÈS")
print("="*80)


RÉSUMÉ DU MODELING AVEC GRIDSEARCHCV

1. MODÈLES ENTRAÎNÉS:
   - Nombre de types de modèles: 6
   - Nombre de configurations de features: 4
   - Total de combinaisons testées: 20
   - Modèles ignorés pour 'Combined': 3

2. OPTIMISATION GRIDSEARCHCV:
   - Cross-validation: 3-fold
   - Métrique d'optimisation: F1-Score weighted
   - Hyperparamètres optimisés pour chaque modèle: Oui

3. MEILLEUR MODÈLE:
   - Nom: Random_Forest
   - Features: Combined
   - CV F1-Score: 0.7085
   - Test F1-Score (weighted): 0.7556
   - Test Accuracy: 0.7663

4. TOP 3 MODÈLES:
   19. Random_Forest_Combined - F1: 0.7556
   10. Random_Forest_Count - F1: 0.7510
   7. Logistic_Regression_Count - F1: 0.7405

5. FICHIERS SAUVEGARDÉS:
   - modeling_results_gridsearch.pkl (tous les modèles et résultats)
   - model_comparison_gridsearch.csv (tableau comparatif)
   - best_hyperparameters.csv (meilleurs hyperparamètres)

6. AMÉLIORATION vs MODÈLES DE BASE:
   - GridSearchCV a optimisé les hyperparamètres
   - Cross-va